In [1]:
from __future__ import unicode_literals

import sys
import os
from bs4 import BeautifulSoup
import urllib2
import cv2
from matplotlib import pyplot as plt
import matplotlib
import threading
import time
import Queue
from PIL import Image, ImageDraw, ImageFont

In [2]:
# from https://stackoverflow.com/questions/4460921/extract-the-first-paragraph-from-a-wikipedia-article-python
import re
import textwrap
import yaml
import urllib
import urllib2
import random

import wikipedia

In [3]:
# set utf-8
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde
sys.getdefaultencoding()

'utf-8'


In [4]:
def de(s):
    print s

In [5]:
OUTPUT_PATH = '/home/ziweihe/sdb1/cvdata/wikipages500_2_samesize/'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [6]:
DATA_SIZE = 500
IMG_SIZE = (227, 227)

In [7]:
LANG = {
    'en': 'https://en.wikipedia.org',
    'zh': 'https://zh.wikipedia.org'
}

In [8]:
class WikipediaError(Exception):
    pass

In [9]:
class Wikipedia:
    url_article = 'http://%s.wikipedia.org/w/index.php?action=raw&title=%s'
    url_image = 'http://%s.wikipedia.org/w/index.php?title=Special:FilePath&file=%s'
    url_search = 'http://%s.wikipedia.org/w/api.php?action=query&list=search&srsearch=%s&sroffset=%d&srlimit=%d&format=yaml'
   
    def __init__(self, lang):
        self.lang = lang
   
    def __fetch(self, url):
        request = urllib2.Request(url)
        request.add_header('User-Agent', 'Mozilla/5.0')
       
        try:
            result = urllib2.urlopen(request)
        except urllib2.HTTPError, e:
            raise WikipediaError(e.code)
        except urllib2.URLError, e:
            raise WikipediaError(e.reason)
       
        return result
   
    def article(self, article):
        url = self.url_article % (self.lang, urllib.quote_plus(article))
        content = self.__fetch(url).read()
       
        if content.upper().startswith('#REDIRECT'):
            match = re.match('(?i)#REDIRECT \[\[([^\[\]]+)\]\]', content)
           
            if not match == None:
                return self.article(match.group(1))
           
            raise WikipediaError('Can\'t found redirect article.')
       
        return content
   
    def image(self, image, thumb=None):
        url = self.url_image % (self.lang, image)
        result = self.__fetch(url)
        content = result.read()
       
        if thumb:
            url = result.geturl() + '/' + thumb + 'px-' + image
            url = url.replace('/commons/', '/commons/thumb/')
            url = url.replace('/' + self.lang + '/', '/' + self.lang + '/thumb/')
           
            return self.__fetch(url).read()
       
        return content
   
    def search(self, query, page=1, limit=10):
        offset = (page - 1) * limit
        url = self.url_search % (self.lang, urllib.quote_plus(query), offset, limit)
        content = self.__fetch(url).read()
       
        parsed = yaml.load(content)
        search = parsed['query']['search']
       
        results = []
       
        if search:
            for article in search:
                title = article['title'].strip()
               
                snippet = article['snippet']
                snippet = re.sub(r'(?m)<.*?>', '', snippet)
                snippet = re.sub(r'\s+', ' ', snippet)
                snippet = snippet.replace(' . ', '. ')
                snippet = snippet.replace(' , ', ', ')
                snippet = snippet.strip()
               
                wordcount = article['wordcount']
               
                results.append({
                    'title' : title,
                    'snippet' : snippet,
                    'wordcount' : wordcount
                })
       
        # yaml.dump(results, default_style='', default_flow_style=False,
        #     allow_unicode=True)
        return results

In [10]:
def unwiki(wiki):
    """
   Remove wiki markup from the text.
   """
    wiki = re.sub(r'(?i)\{\{IPA(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
    wiki = re.sub(r'(?i)\{\{Lang(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
    wiki = re.sub(r'\{\{[^\{\}]+\}\}', '', wiki)
    wiki = re.sub(r'(?m)\{\{[^\{\}]+\}\}', '', wiki)
    wiki = re.sub(r'(?m)\{\|[^\{\}]*?\|\}', '', wiki)
    wiki = re.sub(r'(?i)\[\[Category:[^\[\]]*?\]\]', '', wiki)
    wiki = re.sub(r'(?i)\[\[Image:[^\[\]]*?\]\]', '', wiki)
    wiki = re.sub(r'(?i)\[\[File:[^\[\]]*?\]\]', '', wiki)
    wiki = re.sub(r'\[\[[^\[\]]*?\|([^\[\]]*?)\]\]', lambda m: m.group(1), wiki)
    wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', lambda m: m.group(1), wiki)
    wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', '', wiki)
    wiki = re.sub(r'(?i)File:[^\[\]]*?', '', wiki)
    wiki = re.sub(r'\[[^\[\]]*? ([^\[\]]*?)\]', lambda m: m.group(1), wiki)
    wiki = re.sub(r"''+", '', wiki)
    wiki = re.sub(r'(?m)^\*$', '', wiki)

    return wiki

def unhtml(html):
    """
   Remove HTML from the text.
   """
    html = re.sub(r'(?i)&nbsp;', ' ', html)
    html = re.sub(r'(?i)<br[ \\]*?>', '\n', html)
    html = re.sub(r'(?m)<!--.*?--\s*>', '', html)
    html = re.sub(r'(?i)<ref[^>]*>[^>]*<\/ ?ref>', '', html)
    html = re.sub(r'(?m)<.*?>', '', html)
    html = re.sub(r'(?i)&amp;', '&', html)

    return html

def punctuate(text):
    """
   Convert every text part into well-formed one-space
   separate paragraph.
   """
    text = re.sub(r'\r\n|\n|\r', '\n', text)
    text = re.sub(r'\n\n+', '\n\n', text)

    parts = text.split('\n\n')
    partsParsed = []

    for part in parts:
        part = part.strip()

        if len(part) == 0:
            continue

        partsParsed.append(part)

    return '\n\n'.join(partsParsed)


In [11]:
# class worker_thread (threading.Thread):
#     def __init__(self, threadID, name, func, args):
#         threading.Thread.__init__(self)
#         self.threadID = threadID
#         self.name = name
#         self.func = func
#         self.args = args
#         self._stop_event = threading.Event()
        
#     def run(self):
#         de("Starting " + self.name)
#         self.func(self.threadID, self.args)
#         de("Exiting " + self.name)
        
#     def stop(self):
#         self._stop_event.set()

#     def stopped(self):
#         return self._stop_event.is_set()

In [12]:
atc_set = set([])
atc_lock = threading.Lock()
link_set = set([])
link_lock = threading.Lock()

threads = []

In [13]:
# def get_urls_task(id, url):
#     '''get urls in url and add to lists'''
#     try:
#         content = urllib.urlopen(url)
#     except:
#         return
#     soup = BeautifulSoup(content, "html.parser")
#     base=soup.find('div', id="bodyContent")
    
#     for prefix_key in LANG:
#         if LANG[prefix_key] in url:
#             prefix = LANG[prefix_key]

#     for link in BeautifulSoup(str(base), "html.parser").findAll("a"):
#         if 'href' in link.attrs:  # Make sure it is link
#             if link['href'].startswith('/wiki/') and prefix + link['href'] != url:
#                 link_lock.acquire()
#                 link_set.add(prefix + link['href'])
#                 link_lock.release()
#             if ( 
#                 link['href'].startswith('/wiki/') and  # Make sure it is a wiki page
#                 len(link['href'].split('/')) == 3 and  # Make sure it is an article
#                 len(link['href'].split(':')) == 1      # Make sure it is not category
#             ):
# #                 print(link['href'])
#                 title = str(link['href'])
#                 title = urllib.unquote(title).decode('utf8')
#                 print title
#                 atc_lock.acquire()
#                 if len(atc_set) < DATA_SIZE:
#                     atc_set.add(prefix + title)
#                 atc_lock.release()

In [14]:
# link_set.add(LANG['en'])
# thread_id = 1
# while len(atc_set) < DATA_SIZE:
#     while len(link_set) == 0:
#         pass
#     url = link_set.pop()
#     thread = worker_thread(thread_id, url, get_urls_task, url)
#     thread.start()
#     threads.append(thread)
#     thread_id += 1
#     time.sleep(1)

In [15]:
# link_set_en = link_set
# link_set = set([])

# atc_set_en = atc_set
# atc_set = set([])

In [16]:
# link_set.add(LANG['zh'])
# thread_id = 1
# while len(atc_set) < DATA_SIZE:
#     while len(link_set) == 0:
#         pass
#     url = link_set.pop()
#     thread = worker_thread(thread_id, url, get_urls_task, url)
#     thread.start()
#     threads.append(thread)
#     thread_id += 1
#     time.sleep(1)

In [17]:
# for i in atc_set_en:
#     atc_set.add(i)
# print len(atc_set)

In [18]:
def article_urls_crawl(prefix, root_url, count):
    atc_list = set([])
    link_list = set([root_url])
    while True:
        url = random.choice(tuple(link_list))
        try:
            content = urllib.urlopen(url)
        except:
            continue
#         print urllib.urlopen(url).headers.getheader('Content-Type')
        soup = BeautifulSoup(content, "html.parser")
        base=soup.find('div', id="bodyContent")

        for link in BeautifulSoup(str(base), "html.parser").findAll("a"):
            if 'href' in link.attrs:  # Make sure it is link
                if link['href'].startswith('/wiki/') and prefix + link['href'] != url:
                    link_list.add(prefix + link['href'])
                if ( 
                    link['href'].startswith('/wiki/') and  # Make sure it is a wiki page
                    len(link['href'].split('/')) == 3 and  # Make sure it is an article
                    len(link['href'].split(':')) == 1      # Make sure it is not category
                ):
#                     print(link['href'])
                    title = str(link['href'])
                    title = urllib.unquote(title).decode('utf8')
                    print title
                    atc_list.add(prefix + title)
                    if len(atc_list) == count:
                        print('fetched ' + str(count) + ' article urls from ' + prefix + '\n')
                        return atc_list

urls_by_lang = {}

for key in LANG:
    urls_by_lang[key] = article_urls_crawl(LANG[key], LANG[key], DATA_SIZE)

/wiki/Wikipedia
/wiki/Free_content
/wiki/Encyclopedia
/wiki/English_language
/wiki/Claude_Debussy
/wiki/Impressionism_in_music
/wiki/Conservatoire_de_Paris
/wiki/Pelléas_et_Mélisande_(opera)
/wiki/Béla_Bartók
/wiki/Olivier_Messiaen
/wiki/George_Benjamin_(composer)
/wiki/Bill_Evans
/wiki/Claude_Debussy
/wiki/Intimacy_(Bloc_Party_album)
/wiki/Amy_Adams
/wiki/Oxalaia
/wiki/Joshua_Reynolds
/wiki/David_Garrick_Between_Tragedy_and_Comedy
/wiki/Distracted_boyfriend_meme
/wiki/Nasr,_Sultan_of_Granada
/wiki/Muhammad_III,_Sultan_of_Granada
/wiki/Conchoderma_virgatum
/wiki/Spot-fin_porcupinefish
/wiki/Donald_Trump
/wiki/Kelvin_Droegemeier
/wiki/Office_of_Science_and_Technology_Policy
/wiki/The_Little_Nigar
/wiki/Cakewalk
/wiki/Method_(music)#Piano
/wiki/Rie_Murakawa
/wiki/Westbourne_Terrace
/wiki/Gavin_Buckley
/wiki/Annapolis,_Maryland
/wiki/N._K._Jemisin
/wiki/Hugo_Award_for_Best_Novel
/wiki/The_Stone_Sky
/wiki/Secretary-General_of_the_United_Nations
/wiki/Nobel_Peace_Prize
/wiki/Kofi_Annan
/wik

/wiki/自由內容
/wiki/米歇尔·阿弗拉克
/wiki/叙利亚
/wiki/哲學家
/wiki/社会学
/wiki/阿拉伯民族主义
/wiki/中產階級
/wiki/共产主义
/wiki/社会主义
/wiki/叙利亚－黎巴嫩共产党
/wiki/阿拉伯復興運動
/wiki/阿拉伯復興社會黨
/wiki/埃及總統列表
/wiki/贾迈勒·阿卜杜-纳赛尔
/wiki/阿拉伯聯合共和國
/wiki/黎巴嫩
/wiki/伊拉克
/wiki/阿拉伯世界
/wiki/阿拉伯民族主义
/wiki/资本主义
/wiki/共产主义
/wiki/卡尔·马克思
/wiki/辩证唯物主义
/wiki/阿拉伯社会主义
/wiki/政教分离
/wiki/世俗化
/wiki/无神论
/wiki/基督徒
/wiki/伊斯兰教
/wiki/千禧年大獎難題
/wiki/1850年大西洋飓风季
/wiki/1911年大西洋飓风季
/wiki/诺贝尔文学奖得主列表
/wiki/Unicode中的西里爾字母
/wiki/诺贝尔生理学或医学奖得主列表
/wiki/試播集_(水行俠)
/wiki/DC漫畫
/wiki/环颈潜鸭
/wiki/中国国际救援队
/wiki/中华人民共和国
/wiki/吉隆坡市政局
/wiki/吉隆坡
/wiki/臺灣
/wiki/南投縣
/wiki/埔里鎮
/wiki/埔里茄苳樹王公
/wiki/神木
/wiki/特技摩托賽HD
/wiki/RedLynx
/wiki/向忠发
/wiki/湖北省
/wiki/汉阳府
/wiki/汉川市
/wiki/上海市
/wiki/中国共产党中央政治局
/wiki/中国共产党中央委员会总书记
/wiki/中国共产党
/wiki/中共中央主要负责人
/wiki/二七大罷工
/wiki/中国共产党中央政治局
/wiki/中国共产党
/wiki/冰岛
/wiki/间歇泉
/wiki/穆罕默德六世_(摩洛哥)
/wiki/徵兵制
/wiki/摩洛哥议会
/wiki/尼古拉斯·马杜罗
/wiki/委內瑞拉玻利瓦
/wiki/石油币
/wiki/喀拉拉邦
/wiki/2018年北印度洋氣旋季
/wiki/雅加達
/wiki/2018年亞洲運動會
/wiki/2018年亚洲运动会开幕式
/wiki/諾貝爾和平獎
/wiki/聯合國
/wiki/科菲·安南


In [19]:
def getChinese(context):
    context = context.decode("utf-8") # convert context from str to unicode
    filtrate = re.compile(u'[^\u4E00-\u9FA5]') # non-Chinese unicode range
    context = filtrate.sub(r'', context) # remove all non-Chinese characters
    context = context.encode("utf-8") # convert unicode back to str
    return context

In [20]:
print "Заглавная_страница"

Заглавная_страница


In [21]:
# fontsizerange = (15, 50)
# def gen_img_task(id, url):
#     de("generating image for " + url)
#     for prefix_key in LANG:
#         if LANG[prefix_key] in url:
#             key = prefix_key
    
#     if key is None:
#         return
    
#     de("detected language " + key)
    
#     wikipedia.set_lang(key)
#     res = None
#     try:
#         res = wikipedia.summary(url.split('/')[-1])
#     except:
#         return

# #     text = res
#     text = ""
#     if key == 'en':
#         for n in re.findall(ur'[a-zA-Z ]+',res):
#             text = text + n
#     elif key == 'zh':
#         for n in re.findall(ur'[\u4e00-\u9fff]+',res):
#             text = text + n
# #     de("printing " + text)

#     fontsize = random.randint(fontsizerange[0], fontsizerange[1])
#     fontwidth = IMG_SIZE[1]/(fontsize/3*4)*3

#     image = Image.new('L', IMG_SIZE, 255)
#     draw = ImageDraw.Draw(image)
#     if key == 'en':
#         font = ImageFont.truetype('/usr/share/fonts/dejavu/DejaVuSans.ttf', fontsize)
#         margin = 0
#         offset = 0
#         for line in textwrap.wrap(text, width=fontwidth):
#             draw.text((margin, offset), line, font=font, fill="#000000")
#             offset += font.getsize(line)[1]
#     elif key == 'zh':
#         font = ImageFont.truetype('/home/ziweihe/fonts/msyh.ttf', fontsize)
#         margin = 0
#         offset = 0
#         for line in textwrap.wrap(text, width=fontwidth):
#             draw.text((margin, offset), line, font=font, fill="#000000")
#             offset += font.getsize(line)[1]
#     de("saving " + OUTPUT_PATH + key + '-' + str(id) + '-wiki.jpg')
#     image.save(OUTPUT_PATH + key + '-' + str(id) + '-wiki.jpg')

In [22]:
# thread_id = 1
# while len(atc_set) > 0:
#     url = atc_set.pop()
#     thread = worker_thread(thread_id, url, gen_img_task, url)
#     thread.start()
#     threads.append(thread)
#     thread_id += 1

In [23]:
def generate_imgs(langs, urls_by_lang, count, fontsizerange):
    for key in langs:
        print 'generating imgs for ' + key
        i = 0
        while i < count:
            wikipedia.set_lang(key)
            res = None
            try:
                res = wikipedia.summary(random.choice(tuple(urls_by_lang[key])).split('/')[-1])
            except:
                continue
            
            text = ""
            if key == 'en':
                for n in re.findall(ur'[a-zA-Z ]+',res):
                    text = text + n
            elif key == 'zh':
                for n in re.findall(ur'[\u4e00-\u9fff]+',res):
                    text = text + n
#             print text


            fontsize = random.randint(fontsizerange[0], fontsizerange[1])
            fontwidth = IMG_SIZE[1]/(fontsize/3*4)*3

            image = Image.new('L', IMG_SIZE, 255)
            draw = ImageDraw.Draw(image)
            if key == 'en':
                font = ImageFont.truetype('/usr/share/fonts/dejavu/DejaVuSans.ttf', random.randint(fontsizerange[0], fontsizerange[1]))
                margin = 0
                offset = 0
                for line in textwrap.wrap(text, width=fontwidth):
                    draw.text((margin, offset), line, font=font, fill="#000000")
                    offset += font.getsize(line)[1]
            elif key == 'zh':
                font = ImageFont.truetype('/home/ziweihe/fonts/msyh.ttf', random.randint(fontsizerange[0], fontsizerange[1]))
                margin = 0
                offset = 0
                for line in textwrap.wrap(text, width=fontwidth):
                    draw.text((margin, offset), line, font=font, fill="#000000")
                    offset += font.getsize(line)[1]
            image.save(OUTPUT_PATH + key + '-' + str(i) + '-wiki.jpg')
            i += 1

In [24]:
generate_imgs(LANG, urls_by_lang, DATA_SIZE, (60, 60))

generating imgs for en


/home/ziweihe/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/ziweihe/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


generating imgs for zh
